In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import re

In [2]:
def filechecker(name):
    data=pd.read_csv(name)
    print('top 5 of',len(data),'rows from',name, 'data\n',data.head())

In [3]:
import _thread
import threading

In [4]:
item_link=[]
def swg_crawler(link,filename='swg_brief.csv'):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    
    games=soup.find_all('div',{'class':'cell'})
    games
    for game in games:
        describe=game.find('div',{'class':'h6 name'}).text.strip()

        try:
            describe.split('\n')[0]
            name=describe.split('\n')[0]
        except:
            name=[]
            
        try:
            describe.split('\n')[1]
            discount=describe.split('\n')[1]
        except:
            discount=[]
            
        try:
            describe.split('\n')[2]
            tag=describe.split('\n')[2]
        except:
            tag=[]
            
        try:
            original_price=game.find('s',{'class':'text-muted'}).text.strip()
        except:
            original_price=[]
            
        try:
            discount_price=game.find('strong').text.strip()
        except:
            discount_price=[]
            
        try:
            deadline=game.find('div',{'class':'w-100'}).find('small').text.strip()
        except:
            deadline=[]
            
        
        link=game.find('a',{'class':'main-link'})['href']
        global item_link
        item_link.append('https://www.dekudeals.com'+link)
        
        with open(filename,"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow([name,discount,tag,original_price,discount_price,deadline])
        

In [5]:
def swg_item_crawler(link,filename='swg_detail.csv'):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    name=soup.find('span',{'class':'display-5'}).text
    #name
    info=soup.find_all('li',{'class':'list-group-item'})
    genre=[]
    publisher=[]
    size=[]
    release_date=[]
    metascore=[]
    userscore=[]
    language=[]
    play_mode=[]
    for i in info:
  
        if 'Genre' in i.text:
            genre= i.text.split(': ')[1:]
            #print(genre)
        if 'Publisher' in i.text:
            publisher=i.text.split(': ')[1:]
            #print(publisher)
        if 'Download' in i.text:
            size=i.text.split(': ')[1:]
            #print(size)
        if 'Release' in i.text:
            release_date=i.text.split(': ')[1:]
            #print(release_date)
        if 'Metacritic' in i.text:
            metascore=i.text.split(' ')[1]
            userscore=i.text.split(' ')[2]
            #print(metascore)
            #print(userscore)
        if 'Language' in i.text:
            language=i.text.split(': ')[1:]
            #print(language)
        if 'Play modes' in i.text:
            play_mode=i.text.split(': ')[1:]
            #print(play_mode)
    stores=soup.find_all('tr',{'class':'table-primary'})
    loc={}
    for store in stores:
        try:
            store_loc=store.find('img')['alt']
            version=store.find('td',{'class':'version'}).text.strip()
            price,discount=(store.find('div',{'class','btn-primary'}).text.strip().split('\n'))
            loc.update({store_loc:[version,price,discount]})
        except:
            pass
    #print(loc)
    with open(filename,"a+",encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow([name,genre,publisher,size,release_date,metascore,userscore,language,play_mode,loc])

In [6]:
with open('swg_brief.csv',"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(['Name','Discount','Tag','Original_Price','Discount_Price','Deadline'])

with open('swg_detail.csv',"a+",encoding='utf-8') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(['Name','Genre','Tag','Publisher','Download_Size','Release_Date','MetaScroe','UserScroe','Language','Play_Modes'])

In [7]:
#get the largest page number
link='https://www.dekudeals.com/hottest?page=1'
r=requests.get(link)
soup=BeautifulSoup(r.text,'html5lib')
maxpage=int(soup.find_all('a',{'class':'page-link'})[-2].text)+1
maxpage

12

In [8]:
main='https://www.dekudeals.com/'
link_list=[]
for i in range(1,maxpage):
    link_list.append(main+'hottest?page='+str(i))
link_list

['https://www.dekudeals.com/hottest?page=1',
 'https://www.dekudeals.com/hottest?page=2',
 'https://www.dekudeals.com/hottest?page=3',
 'https://www.dekudeals.com/hottest?page=4',
 'https://www.dekudeals.com/hottest?page=5',
 'https://www.dekudeals.com/hottest?page=6',
 'https://www.dekudeals.com/hottest?page=7',
 'https://www.dekudeals.com/hottest?page=8',
 'https://www.dekudeals.com/hottest?page=9',
 'https://www.dekudeals.com/hottest?page=10',
 'https://www.dekudeals.com/hottest?page=11']

In [9]:
threads=[]
#build all the tasks 
for link in link_list:
    thread = threading.Thread(target=swg_crawler,args=(link,'swg_brief.csv'))
    threads.append(thread) 
 
                              
startTime=time.time()                              
#start all the tasks at once
for thread in threads:
    thread.start()
    
#wait for all the tasks to finish
for thread in threads:
    thread.join()
                              
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} pages'.format(len(link_list)))

      
filechecker('swg_brief.csv')


4.10225772857666 secs spend to craw these 11 pages
top 5 of 388 rows from swg_brief.csv data
                                       Name Discount                   Tag  \
0          The Escapists: Complete Edition     -80%  Matches previous low   
1                      Super Mario Odyssey     -17%                    []   
2                                Hard West     -90%  Matches previous low   
3  The Legend of Zelda: Breath of the Wild     -18%                    []   
4                   Katamari Damacy REROLL     -33%                    []   

  Original_Price Discount_Price          Deadline  
0         $14.99          $2.99  Sale ends May 24  
1         $59.99         $49.99                []  
2         $19.99          $1.99  Sale ends May 27  
3         $59.99         $49.49                []  
4         $29.99         $19.99                []  


In [10]:
import _thread

In [13]:
startTime=time.time()     
n=0
for link in item_link:
    n=n+1
    _thread.start_new_thread(swg_item_crawler, (link, ) )
    if n%50==0:
        n=0
        time.sleep(5)

                                  
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} games'.format(len(item_link)))

filechecker('swg_detail.csv')

35.039695024490356 secs spend to craw these 388 games
top 5 of 785 rows from swg_detail.csv data
                                                 Name  \
0              Night Trap - 25th Anniversary Edition   
1        Mario + Rabbids Kingdom Battle Gold Edition   
2  Baldur's Gate and Baldur's Gate II: Enhanced E...   
3                                      Blazing Beaks   
4                              Degrees of Separation   

                                          Genre                    Tag  \
0                            ['Action, Puzzle']  ['Limited Run Games']   
1                       ['Adventure, Strategy']            ['Ubisoft']   
2                   ['Role-Playing, Adventure']            ['Beamdog']   
3  ['Action, Multiplayer, Party, Role-Playing']         ['QubicGames']   
4           ['Platformer, Puzzle, Multiplayer']        ['Modus Games']   

      Publisher                                      Download_Size  \
0    ['6.2 GB']                                ['A

In [14]:
startTime=time.time()     
for link in item_link:
    swg_item_crawler(link,'swg_detail_single.csv')
finishTime = time.time()
print(finishTime - startTime,'secs spend to craw these {0} games'.format(len(item_link)))

filechecker('swg_detail_single.csv')

157.57485103607178 secs spend to craw these 388 games
top 5 of 387 rows from swg_detail_single.csv data
            The Escapists: Complete Edition  \
0                      Super Mario Odyssey   
1                                Hard West   
2  The Legend of Zelda: Breath of the Wild   
3                   Katamari Damacy REROLL   
4                              Flat Heroes   

         ['Role-Playing, Simulation, Puzzle']                      ['Team17']  \
0                      ['Platformer, Action']                    ['Nintendo']   
1     ['Adventure, Simulation, Role-Playing']       ['Forever Entertainment']   
2         ['Adventure, Action, Role-Playing']                    ['Nintendo']   
3                                  ['Puzzle']  ['BANDAI NAMCO Entertainment']   
4  ['Action, Application, Party, Platformer']            ['Parallel Circles']   

    ['455 MB']  ['NA/EU', 'September 25, 2018JP', 'May 23, 2019']   67  6.0  \
0   ['5.2 GB']                               ['Octob